In [15]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
df=pd.read_csv("locations.csv",index_col='Users')
items=pd.read_csv("warehouses.csv")

In [17]:
df.head(1)

,Urban,Semi_Urban,Rural,Semi_Rural,Remote
Users,,,,,
70-262-7631,18699.0,NaN,NaN,95878.0,802714.0


In [18]:
df=df.fillna(0)

In [19]:
df['Urban'] = df['Urban'].astype('int')
df['Semi_Urban'] = df['Semi_Urban'].astype('int')
df['Rural'] = df['Rural'].astype('int')
df['Semi_Rural']=df['Semi_Rural'].astype('int')
df['Remote']=df['Remote'].astype('int')


In [20]:
total = df['Urban'] + df['Semi_Urban']+df['Rural']+df['Semi_Rural']+df['Remote']
df = df.pipe(lambda x: (x*100).div(total, axis='index'))


In [21]:
df

,Urban,Semi_Urban,Rural,Semi_Rural,Remote
Users,,,,,
70-262-7631,2.038503,0.000000,0.000000,10.452299,87.509198
67-529-8002,0.000000,0.000000,100.000000,0.000000,0.000000
71-032-7622,5.801065,31.240757,0.000000,62.958177,0.000000
57-852-3909,9.889037,32.302924,57.808038,0.000000,0.000000
52-706-8422,1.439550,3.752258,0.000000,7.998960,86.809231
...,...,...,...,...,...
19-969-8322,2.945913,0.000000,0.000000,0.000000,97.054087
97-991-2766,38.004776,61.995224,0.000000,0.000000,0.000000
36-333-1725,NaN,NaN,NaN,NaN,NaN


In [22]:
df=df.fillna(0)

In [23]:
def standardize(row):
    new_row = (row - row.mean())/(row.max()-row.min())
    return new_row

df_std = df.apply(standardize).T
print(df_std)

sparse_df = sparse.csr_matrix(df_std.values)
corrMatrix = pd.DataFrame(cosine_similarity(sparse_df),index=df.columns,columns=df.columns)
corrMatrix

Users       70-262-7631  67-529-8002  71-032-7622  57-852-3909  52-706-8422  \
Urban         -0.022069    -0.042454     0.015556     0.056436    -0.028059   
Semi_Urban    -0.120431    -0.120431     0.191976     0.202598    -0.082909   
Rural         -0.206394     0.793606    -0.206394     0.371686    -0.206394   
Semi_Rural    -0.081998    -0.186521     0.443061    -0.186521    -0.106532   
Remote         0.438893    -0.436199    -0.436199    -0.436199     0.431893   

Users       44-801-7398  32-006-4001  51-912-0166  31-892-4916  38-876-1221  \
Urban         -0.039124    -0.023399    -0.042454     0.010580    -0.018918   
Semi_Urban    -0.080530    -0.120431     0.037467     0.328266     0.159467   
Rural         -0.206394    -0.078895     0.205879     0.291874    -0.206394   
Semi_Rural    -0.186521    -0.186521     0.243307    -0.186521     0.510044   
Remote         0.520569     0.417247    -0.436199    -0.436199    -0.436199   

Users       ...  43-694-1884  60-681-3620  78-028-

,Urban,Semi_Urban,Rural,Semi_Rural,Remote
Urban,1.000000,0.075644,-0.002146,-0.061744,-0.291059
Semi_Urban,0.075644,1.000000,0.000284,-0.039855,-0.462501
Rural,-0.002146,0.000284,1.000000,-0.071725,-0.571299
Semi_Rural,-0.061744,-0.039855,-0.071725,1.000000,-0.510559
Remote,-0.291059,-0.462501,-0.571299,-0.510559,1.000000


In [24]:
def get_similar(movie_name,rating):
    similar_score = corrMatrix[movie_name]*(rating-50.5) #50..5 is the avg of percentage (0-100)
    similar_score = similar_score.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_score
get_similar("Remote",2)

Rural         27.707995
Semi_Rural    24.762088
Semi_Urban    22.431296
Urban         14.116362
Remote       -48.500000
Name: Remote, dtype: float64

In [26]:
warehouse_investor = [("Rural",2.038503),("Semi_Rural",0),("Semi_Urban",0),("Urban",10.452299)]
similar_scores = pd.DataFrame()
for warehouse,percentage_portfolio in warehouse_investor:
    similar_scores = similar_scores.append(get_similar(warehouse,percentage_portfolio),ignore_index = True)

similar_scores.head(10)

/var/folders/yg/rsw8j92510j7cjvvkgjr4s2r0000gn/T/ipykernel_5501/3906771805.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_scores = similar_scores.append(get_similar(warehouse,percentage_portfolio),ignore_index = True)


,Remote,Semi_Rural,Urban,Semi_Urban,Rural
0,27.685998,3.475925,0.103984,-0.013768,-48.461497
1,25.783205,-50.500000,3.118065,2.012695,3.622137
2,23.356297,2.012695,-3.820037,-50.500000,-0.014347
3,11.656244,2.472699,-40.047701,-3.029380,0.085930


In [27]:
similar_scores.sum().sort_values(ascending=False)[:2]

Remote    88.481744
Urban    -40.645690
dtype: float64